### Why Python?

- free, open source
- one platform for data pre-processing, visualization and analysis
- reproducible code
- large number of user-developed packages (eg. nibabel, nilearn)
- easy interaction with state-of-the art neuroimaging software (eg. FSL, ANTS)

### Types of MR Scans

<img src="../static/images/mr_scan_types.png" alt="Drawing" align="middle" width="500px"/>

#### Structural
Structural MRI scans ...

#### Functional
Functional MRI scans...

For this tutorial, we'll be focusing on T1w and resting state fMRI scans.

### Neuroimaging File Formats

|Format Name | File Extension | Origin |
|---|---|---|
| Analyze | .img/.hdr | Analyze Software, Mayo Clinic |
| DICOM | none | ACR/NEMA Consortium |
| NIfTI | .nii or .img/.hdr | Neuroimaging Informatics Technology Initiative |
| MINC | .mnc | Montreal Neurological Institute |
| NRRD | .nrrd | |

<img src="../static/images/dicom_to_nifti.png" alt="Drawing" align="middle" width="300px"/>

From the MRI scanner, images are initially collected in the DICOM format and can be converted to NIfTI using [dcm2niix](https://github.com/rordenlab/dcm2niix).

### Intro to NIfTI

NIfTI is one of the most common file formats for storing neuroimaging data. We'll cover a few details to get started working with them. If you're interested in learning more about NIfTI images, we highly recommend [this blog post about the NIfTI format](http://brainder.org/2012/09/23/the-nifti-file-format/).

### Reading NIfTI Images

[NiBabel](http://nipy.org/nibabel/) is a Python package for reading and writing neuroimaging data. To learn more about how NiBabel handles NIfTIs, check out the [Working with NIfTI images](http://nipy.org/nibabel/nifti_images.html) page of the NiBabel documentation.

In [2]:
import nibabel as nib

First, use the `load()` function to create a NiBabel image object from a NIfTI file. We'll load in a T1w image from the dataset we'll be using for this tutorial.

In [3]:
t1_img = nib.load('../data/ds000030/sub-10206/anat/sub-10206_T1w.nii.gz')

There are three main components of a NIfTI image:

#### 1. [Header](http://nipy.org/nibabel/nibabel_images.html#the-image-header): contains metadata about the image, such as image dimensions, data type, etc.

In [ ]:
nii_hdr = t1_img.header

You can easily accessing specific metadata from the NiBabel image header object through dictionary keys.

In [ ]:
nii_hdr.keys()

In [ ]:
nii_hdr['pixdim']

#### 2. [Affine](http://nipy.org/nibabel/coordinate_systems.html): tells the position of the image array data in a *reference space*

The affine array tells the position of the image array data in a *reference space*. It translates between data-space and world-space.

In [ ]:
nii_affine = t1_img.affine
nii_affine

In [ ]:
# go from voxel to mm
real_pt = nib.affines.apply_affine(nii_affine, [22, 34, 12])
real_pt

In [ ]:
# go from mm to voxel
import numpy.linalg as npl
nib.affines.apply_affine(npl.inv(nii_affine), real_pt)

ADD MORE DETAIL!

#### 3. Data

The data is a multidimensional array representing the image data.

In [ ]:
nii_data = t1_img.get_data()
nii_data

#### Explore

Explore some of the other methods that can be called on the NIfTI image. Can you get the dimensions of the image? The data type?

In [ ]:
t1_img.get_data_dtype()

The data type of an image controls the range of possible intensities. As the number of possible values increases, the size of the image also increases.

| Data Type | Range | Number of Values |
|---|---|---|
| uint8 | 0, 255 | 256 |
| uint16 | -128, 127 | 256 |
| uint 16 | 0, 2^16 | 2^16 |
| int16 | -2^15, 2^15 | 2^16 |
| float16 | ~-2^16, ~2^16 | >>2^16 |

The shape of the data always has at least 3 dimensions (X, Y, and Z) and sometimes T (time).

In [ ]:
t1_img.shape

This T1w image has 3 dimensions. The brain was scanned in 176 slices with a resolution of 256 x 256 voxels per slice.

The image has 4 dimensions: x, y, z and time.

### Working With Image Data

#### Slicing

n-dimensional images are just stacks of numpy arrays.

add nibabel slicing 

#### Reshaping

NiBabel has a `reshape()` function for reshaping the data array. Let's say we want to convert this 3D array into a a 2D n_voxels * n_volumes) array.

In [ ]:
nii_data_2d = nii_data.reshape(np.prod(nii_data.shape[:-1]), nii_data.shape[-1])
nii_data_2d.shape

### Coordinate Systems

### Viewing Images

In [ ]:
data = np.rot90(nii_data[:,:,:,0], 1)

In [ ]:
data.shape

In [ ]:
data[:,:,0]

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 6, figsize=[18, 3])

n = 0
slice = 0

for i in range(6):
    axes[n].imshow(data[:, :, slice], cmap='gray')
    axes[n].set_xticks([])
    axes[n].set_yticks([])
    axes[n].set_title('Slice number: {}'.format(slice), color='r')
    n += 1
    slice +=4

fig.subplots_adjust(wspace=0, hspace=0)
plt.show()

Matplotlib's `imshow()` function displays 2D image data. 

We are looking at the subject's brain fromt eh top with slice 0 being the lowest one and slice 20 being the highest one.

In [ ]:
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice, cmap="gray", origin="lower")
        for ax in axes:
            ax.axis('off')

In [ ]:
# view center slice
slice_0 = nii_data[63, :, :, 0]
slice_1 = nii_data[:, 47, :, 0]
slice_2 = nii_data[:, :, 11, 0]
show_slices([slice_0, slice_1, slice_2])
plt.suptitle("Center slices for EPI image")

In this example, 
- the x dimension is in the sagittal plane,
- the y dimension is in the coronal plane
- the z dimension is in the axial plane

Rotate the slices

In [ ]:
show_slices([slice_0.T, slice_1.T, slice_2.T])

### Masks

In [ ]:
import scipy.ndimage as ndi

In [ ]:
slice_3 = slice_2.T

In [ ]:
hist=ndi.histogram(slice_3, min=0, max=255, bins=256)
plt.plot(hist)
plt.show()

In [ ]:
mask = slice_3 > 64

In [ ]:
plt.imshow(mask, cmap = 'gray')

In [ ]:
test = np.where(slice_3 > 80, slice_3, 0)
plt.imshow(test, cmap = 'gray')

### Writing NIfTI Images

Let's save the mask we just created to a file.

In [ ]:
img_mask = nib.Nifti1Image(test, nii_affine, nii_hdr)

In [ ]:
img_mask.to_filename('')